<a href="https://colab.research.google.com/github/psagrera/Data-Science-Advanced/blob/main/DeepLearning/Session2/S2_Worksheet/S2_DL_worksheet_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<img src="https://drive.google.com/uc?id=1zWG2FYCrmlZ8HmPjmgoIZikGW9MbrxMl" style="height: 100px">
<center style="color:#888">Módulo Data Science in IoT<br/>Asignatura Deep Learning</center>

# Worksheet S2: Clasificación con TensorFlow

## Objetivos

El objetivo de este worksheet es familiarizarnos con TensorFlow mediante un primer ejemplo basado en clasificación. 



## Introducción


### Clasificación

En este worksheet vamos a tratar con nuestro primer problema de clasificación en TensorFlow. Mientras que en regresión lo que intentamos era predecir un valor numérico, en clasificación intentamos separar nuestros datos en distintas clases conocidas. En este ejemplo vamos a utilizar un estimador de TensorFlow para clasificar flores.

In [ ]:
import tensorflow as tf

import pandas as pd

### Dataset

El dataset que vamos a utilizar contiene 3 clases distintas:
- Setosa
- Versicolor
- Virginica

De cada flor, se cuenta con la siguiente información:

- Longitud del sépalo
- Anchura del sépalo
- Longitud del pétalo
- Anchura del pétalo

Vamos a definir estas constantes para utilizarlas posteriormente:

In [ ]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

El dataset que vamos a utilizar está preparado en el módulo de Keras para poder ser utilizado con facilidad, podemos cargarlo así:

In [ ]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)


8192/573 [============================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


Como siempre, comenzamos echando un vistazo a los datos:

In [ ]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


A continuación vamos a retirar la columna "Especie" y la utilizaremos como etiqueta:

In [ ]:
train_y = train.pop('Species')
test_y = test.pop('Species')
train.head() # como podemos ver, la especie ya no aparece

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [ ]:
train.shape

(120, 4)

### Input Function

Al igual que hicimos en nuestro ejemplo de regresión, necesitamos una input function. Por suerte, esta es más sencilla que la anterior :)

In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convertimos los datos al tipo dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Aleatorizamos y seleccionamos los datos si estamos en modo train.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

### Feature Columns

Y de nuevo, necesitamos nuestros feature columns

In [ ]:
# Feature columns describe cómo se utilizan las feature columns
my_feature_columns = []
for key in train.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


### Construyendo el Modelo

Lo siguiente que tenemos que hacer es elegir el modelo. Para resolver un problema de clasificación de este tipo hay diferentes estimadores o modelos que podemos utilizar, por ejemplo:

- Clasificador DNN (Deep Neural Network)
- Clasificador Lineal

Vamos a utilizar el estimador basado en una red neuronal, ya que no podemos asumir que haya una correspondencia lineal en nuestro dataset.

Vamos a ello:

In [ ]:
# Definimos una DNN con 2 capas ocultas con 30 y 10 nodos ocultos cada una.
classifier = tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    # Dos capas ocultas de 30 y 10 nodos respectivamente
    hidden_units=[30,10],
    # El modelo debe poder distinguir entre 3 clases
    n_classes=3)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/01/4d70j3nn55z71b4011q5p56r0000gn/T/tmp21oq8rrp', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Como hemos comentado previamente, el número de capas y neuronas ocultas es un número arbitrario y necesitamos de muchos experimentos y test para poder determinar la mejor elección de estos valores.

### Training

Vamos a entrenar nuestra primera red neuronal 

In [ ]:
classifier.train(
    input_fn=lambda: input_fn(train, train_y, training=True),
    steps=40000)
# Hemos definido lambda para evitar crear una función previamente (los detalles de implementación no son importantes ahora)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/01/4d70j3nn55z71b4011q5p56r0000gn/T/tmp21oq8rrp/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /var/folders/01/4d70j3nn55z71b4011q5p56r0000gn/T/tmp21oq8rrp/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.7072174, step = 5000
INFO:tensorflow:global_step/sec: 499.206
INFO:tensorflow:loss = 0.71547776, step = 5100 (0.201 sec)
INFO:tensorflow:global_step/sec: 646.914
INFO:tensorflow:loss = 0.7099789, step = 5200 (0.155 sec)
INFO:tensorflow:global_step/sec: 647.895
INFO:tensor

El único parámetro que no hemos visto hasta ahora es el parámetro steps. Este parámetro le dice al clasificador que actualice el valor de los parámetros libres 5000 veces, cuanto mayor sea el número más datos verá la red.

### Evaluación

Evaluamos nuestro modelo:

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(test, test_y, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-04-15T21:38:46Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/01/4d70j3nn55z71b4011q5p56r0000gn/T/tmp21oq8rrp/model.ckpt-45000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.22091s
INFO:tensorflow:Finished evaluation at 2021-04-15-21:38:47
INFO:tensorflow:Saving dict for global step 45000: accuracy = 1.0, average_loss = 0.18637206, global_step = 45000, loss = 0.18637206
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 45000: /var/folders/01/4d70j3nn55z71b4011q5p56r0000gn/T/tmp21oq8rrp/model.ckpt-45000

Test set accuracy: 1.000



Como podemos ver, ahora no hemos especificado el número de pasos. Esto se debe a que durante la evaluación el modelo símplemente va a ver todos los datos de test una sola vez para poder clasificarlos.

### Predicciones

Una vez entrenado y evaluado el modelo, el último paso que tenemos que hacer es utilizar este modelo para realizar predicciones. A continuación tenemos un pequeño script que nos permite introducir las características de una flor y ver su predicción:

In [ ]:
def input_fn(features, batch_size=256):
    # Transforma las entradas a un Dataset sin etiquetas
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Por favor, introduce valores numéricos de tipo float64 (los números deben ser decimales).")
for feature in features:
  valid = True
  while valid: 
    val = input(feature + ": ")
    if not val.isdigit(): valid = False

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('La predicción es: "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))


Por favor, introduce valores numéricos de tipo float64 (los números deben ser decimales).


SepalLength:  5.1
SepalWidth:  3.3
PetalLength:  1.7
PetalWidth:  0.5


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /var/folders/01/4d70j3nn55z71b4011q5p56r0000gn/T/tmp21oq8rrp/model.ckpt-45000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
La predicción es: "Setosa" (88.8%)


In [ ]:
# Estos son unos cuantos ejemplos que podríamos probar en el script de arriba:
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}